<a href="https://colab.research.google.com/github/Arwa008/AI/blob/main/Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Colab edition by Ghassem Tofighi
#Download and extract all necessary files
!rm -rf /content/*
!wget https://github.com/aimacode/aima-python/archive/master.zip 2>/dev/null
!unzip -q master.zip
!mv aima-python-master/* /content
!wget https://github.com/aimacode/aima-data/archive/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86.zip 2>/dev/null
!unzip -q f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86.zip
!rm -rf aima-data
!mv aima-data-f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86 aima-data
#Install Libraries
!pip install ipythonblocks 2>/dev/null

In [ ]:
from agents import *
class Parcels(Thing):
    pass
class Park2D(XYEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
            else:
                print('{} decided to move {}wards at location: {}, but couldn\'t'.format(str(agent)[1:-1], agent.direction.direction,                                                                            agent.location))
                agent.moveforward(False)
        elif action == "collect":
            items = self.list_things_at(agent.location, tclass=Parcels)
            if len(items) != 0:
                if agent.collect(items[0]):
                    print('{} gethered {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
    def is_done(self):
        no_edibles = not any(isinstance(thing, Parcels) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


In [ ]:
from random import choice
turn = False
class EnergeticCar(Agent):
    location = [0,1]
    direction = Direction("down")
    def moveforward(self, success=True):
        global turn
        if not success:
            turn = True # if edge has been reached, remember to turn
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    def turn(self, d):
        self.direction = self.direction + d
    def collect(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Parcels):
            return True
        return False
def program(percepts):
    '''Returns an action based on it's percepts'''
    global turn
    for p in percepts:
        if isinstance(p, Parcels):
            return 'collect'
    if turn: # then recall if you were at an edge and had to turn
        turn = False
        choice = random.choice((1,2));
    else:
        choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'


In [ ]:
CarEnivronment = Park2D(3,3)
car = EnergeticCar(program)
parcels = Parcels()
CarEnivronment.add_thing(car, [0,1])
CarEnivronment.add_thing(parcels, [1,2])
parcel2 = Parcels()
CarEnivronment.add_thing(parcel2, [1,1])
parcel3 = Parcels()
CarEnivronment.add_thing(parcel3, [2,2])
print("Car started at [0,1], facing down. Let's see if he found any parcels!")
CarEnivronment.run(20)

Car started at [0,1], facing down. Let's see if he found any parcels!
EnergeticCar decided to turnright at location: [0, 1]
EnergeticCar decided to turnright at location: [0, 1]
EnergeticCar decided to turnright at location: [0, 1]
EnergeticCar decided to move rightwards at location: [0, 1]
EnergeticCar gethered Parcels at location: [1, 1]
EnergeticCar decided to turnright at location: [1, 1]
EnergeticCar decided to move downwards at location: [1, 1]
EnergeticCar gethered Parcels at location: [1, 2]
EnergeticCar decided to turnleft at location: [1, 2]
EnergeticCar decided to move rightwards at location: [1, 2]
EnergeticCar gethered Parcels at location: [2, 2]
